![Redis](https://redis.io/wp-content/uploads/2024/04/Logotype.svg?auto=webp&quality=85,75&width=120)

# Context Quarantine: Multi-Agent Isolation

## Learning Objectives (40 minutes)
By the end of this notebook, you will be able to:
1. **Understand** context contamination and why isolation matters
2. **Implement** specialized agents with isolated memory namespaces
3. **Design** agent handoff patterns using LangGraph
4. **Create** focused conversation threads for different tasks
5. **Measure** the benefits of context quarantine on agent performance

## Prerequisites
- Completed previous notebooks in Section 5
- Understanding of LangGraph workflows
- Familiarity with Agent Memory Server namespaces

---

## Introduction

**Context Quarantine** is the practice of isolating different types of conversations and tasks into separate memory spaces to prevent context contamination. Just like medical quarantine prevents disease spread, context quarantine prevents irrelevant information from degrading agent performance.

### The Context Contamination Problem

Without proper isolation, agents suffer from:
- **Topic drift**: Academic planning conversations contaminated by course browsing
- **Preference confusion**: Career advice mixed with course preferences
- **Memory interference**: Irrelevant memories retrieved for current tasks
- **Decision paralysis**: Too much unrelated context confuses the LLM

### Our Solution: Specialized Agent Architecture

We'll create specialized agents for your Redis University system:
1. **CourseExplorerAgent**: Course discovery and browsing
2. **AcademicPlannerAgent**: Degree planning and requirements
3. **CareerAdvisorAgent**: Career guidance and opportunities
4. **PreferenceManagerAgent**: Student preferences and settings

Each agent maintains isolated memory and focused tools.

## Environment Setup

In [ ]:
# Environment setup
import os
import asyncio
import json
from typing import List, Dict, Any, Optional, Union
from dataclasses import dataclass, field
from enum import Enum
from dotenv import load_dotenv
import uuid
from datetime import datetime

# Load environment variables
load_dotenv()

REDIS_URL = os.getenv("REDIS_URL", "redis://localhost:6379")
AGENT_MEMORY_URL = os.getenv("AGENT_MEMORY_URL", "http://localhost:8088")
OPENAI_API_KEY = os.getenv("OPENAI_API_KEY")

print("🔧 Environment Setup")
print("=" * 30)
print(f"Redis URL: {REDIS_URL}")
print(f"Agent Memory URL: {AGENT_MEMORY_URL}")
print(f"OpenAI API Key: {'✅ Set' if OPENAI_API_KEY else '❌ Not set'}")

In [ ]:
# Import required modules
try:
    import redis
    from redis_context_course.models import StudentProfile, Course
    from redis_context_course.course_manager import CourseManager
    from redis_context_course.redis_config import redis_config
    
    # Redis connection
    redis_client = redis.from_url(REDIS_URL)
    if redis_config.health_check():
        print("✅ Redis connection healthy")
    else:
        print("❌ Redis connection failed")
    
    # Course manager
    course_manager = CourseManager()
    
    print("✅ Core modules imported successfully")
    
except ImportError as e:
    print(f"❌ Import failed: {e}")
    print("Please ensure you've completed the setup from previous sections.")

## Agent Specialization Framework

Let's define our specialized agent architecture:

In [ ]:
class AgentType(Enum):
    """Types of specialized agents."""
    COURSE_EXPLORER = "course_explorer"
    ACADEMIC_PLANNER = "academic_planner"
    CAREER_ADVISOR = "career_advisor"
    PREFERENCE_MANAGER = "preference_manager"
    COORDINATOR = "coordinator"  # Routes between agents

@dataclass
class AgentContext:
    """Isolated context for a specialized agent."""
    agent_type: AgentType
    student_id: str
    session_id: str
    memory_namespace: str
    conversation_history: List[Dict[str, Any]] = field(default_factory=list)
    active_tools: List[str] = field(default_factory=list)
    context_data: Dict[str, Any] = field(default_factory=dict)
    
    def add_message(self, role: str, content: str, metadata: Optional[Dict] = None):
        """Add a message to the conversation history."""
        message = {
            "role": role,
            "content": content,
            "timestamp": datetime.now().isoformat(),
            "metadata": metadata or {}
        }
        self.conversation_history.append(message)
    
    def get_recent_context(self, max_messages: int = 10) -> List[Dict[str, Any]]:
        """Get recent conversation context."""
        return self.conversation_history[-max_messages:]

class SpecializedAgent:
    """Base class for specialized agents with isolated context."""
    
    def __init__(self, agent_type: AgentType, student_id: str):
        self.agent_type = agent_type
        self.student_id = student_id
        self.session_id = str(uuid.uuid4())
        self.memory_namespace = f"{agent_type.value}_{student_id}"
        
        # Create isolated context
        self.context = AgentContext(
            agent_type=agent_type,
            student_id=student_id,
            session_id=self.session_id,
            memory_namespace=self.memory_namespace
        )
        
        # Define agent-specific tools and capabilities
        self._setup_agent_capabilities()
    
    def _setup_agent_capabilities(self):
        """Setup agent-specific tools and capabilities."""
        # Override in subclasses
        pass
    
    async def process_query(self, query: str) -> Dict[str, Any]:
        """Process a query within this agent's specialized context."""
        # Add user message to context
        self.context.add_message("user", query)
        
        # Process with agent-specific logic
        response = await self._process_specialized_query(query)
        
        # Add agent response to context
        self.context.add_message("assistant", response["content"])
        
        return response
    
    async def _process_specialized_query(self, query: str) -> Dict[str, Any]:
        """Process query with agent-specific logic. Override in subclasses."""
        return {
            "content": f"[{self.agent_type.value}] Processing: {query}",
            "agent_type": self.agent_type.value,
            "tools_used": [],
            "context_size": len(self.context.conversation_history)
        }
    
    def get_context_summary(self) -> Dict[str, Any]:
        """Get a summary of this agent's context."""
        return {
            "agent_type": self.agent_type.value,
            "memory_namespace": self.memory_namespace,
            "conversation_length": len(self.context.conversation_history),
            "active_tools": self.context.active_tools,
            "context_data_keys": list(self.context.context_data.keys())
        }

print("✅ Agent specialization framework defined")

## Specialized Agent Implementations

Now let's create our specialized agents for the Redis University system:

In [ ]:
class CourseExplorerAgent(SpecializedAgent):
    """Specialized agent for course discovery and browsing."""
    
    def _setup_agent_capabilities(self):
        """Setup course exploration specific tools."""
        self.context.active_tools = [
            "search_courses",
            "get_course_details",
            "filter_by_format",
            "filter_by_difficulty"
        ]
        
        self.context.context_data = {
            "search_history": [],
            "viewed_courses": [],
            "search_filters": {}
        }
    
    async def _process_specialized_query(self, query: str) -> Dict[str, Any]:
        """Process course exploration queries."""
        query_lower = query.lower()
        
        # Track search in context
        self.context.context_data["search_history"].append({
            "query": query,
            "timestamp": datetime.now().isoformat()
        })
        
        if any(word in query_lower for word in ["search", "find", "courses", "classes"]):
            # Simulate course search
            try:
                # Extract search terms
                search_terms = self._extract_search_terms(query)
                results = await course_manager.search_courses(search_terms, limit=3)
                
                if results:
                    # Track viewed courses
                    course_codes = [c.course_code for c in results]
                    self.context.context_data["viewed_courses"].extend(course_codes)
                    
                    course_list = "\n".join([
                        f"• {c.course_code}: {c.title} ({c.format.value}, {c.difficulty.value})"
                        for c in results
                    ])
                    
                    content = f"Found {len(results)} courses matching '{search_terms}':\n{course_list}"
                else:
                    content = f"No courses found for '{search_terms}'. Try different search terms."
                    
                return {
                    "content": content,
                    "agent_type": self.agent_type.value,
                    "tools_used": ["search_courses"],
                    "context_size": len(self.context.conversation_history),
                    "search_results_count": len(results) if results else 0
                }
                
            except Exception as e:
                content = f"I can help you search for courses. What topic interests you?"
        
        elif "details" in query_lower or "about" in query_lower:
            content = "I can provide detailed information about specific courses. Which course would you like to know more about?"
        
        else:
            content = "I'm your course exploration assistant! I can help you search for courses, get course details, and filter by format or difficulty. What would you like to explore?"
        
        return {
            "content": content,
            "agent_type": self.agent_type.value,
            "tools_used": [],
            "context_size": len(self.context.conversation_history)
        }
    
    def _extract_search_terms(self, query: str) -> str:
        """Extract search terms from query."""
        # Simple extraction - in real implementation, use NLP
        stop_words = {"search", "find", "courses", "for", "about", "on", "in", "the", "a", "an"}
        words = query.lower().split()
        search_terms = [word for word in words if word not in stop_words]
        return " ".join(search_terms) if search_terms else "programming"

class AcademicPlannerAgent(SpecializedAgent):
    """Specialized agent for degree planning and academic requirements."""
    
    def _setup_agent_capabilities(self):
        """Setup academic planning specific tools."""
        self.context.active_tools = [
            "check_prerequisites",
            "plan_degree_path",
            "check_graduation_requirements",
            "recommend_next_courses"
        ]
        
        self.context.context_data = {
            "degree_progress": {},
            "planned_courses": [],
            "graduation_timeline": {}
        }
    
    async def _process_specialized_query(self, query: str) -> Dict[str, Any]:
        """Process academic planning queries."""
        query_lower = query.lower()
        
        if any(word in query_lower for word in ["plan", "degree", "graduation", "requirements"]):
            content = "I can help you plan your degree! I'll analyze your completed courses, check requirements, and create a graduation timeline. What's your major and target graduation date?"
            tools_used = ["plan_degree_path"]
        
        elif any(word in query_lower for word in ["prerequisites", "can I take", "ready for"]):
            content = "I'll check if you meet the prerequisites for specific courses. Which course are you interested in taking?"
            tools_used = ["check_prerequisites"]
        
        elif any(word in query_lower for word in ["next", "should take", "recommend"]):
            content = "Based on your academic progress, I can recommend the best courses to take next semester. Let me analyze your completed courses and degree requirements."
            tools_used = ["recommend_next_courses"]
        
        else:
            content = "I'm your academic planning assistant! I can help you plan your degree, check prerequisites, and recommend courses for graduation. What would you like to plan?"
            tools_used = []
        
        return {
            "content": content,
            "agent_type": self.agent_type.value,
            "tools_used": tools_used,
            "context_size": len(self.context.conversation_history)
        }

class CareerAdvisorAgent(SpecializedAgent):
    """Specialized agent for career guidance and opportunities."""
    
    def _setup_agent_capabilities(self):
        """Setup career guidance specific tools."""
        self.context.active_tools = [
            "find_career_paths",
            "recommend_internships",
            "analyze_job_market",
            "suggest_skill_development"
        ]
        
        self.context.context_data = {
            "career_interests": [],
            "explored_paths": [],
            "skill_gaps": []
        }
    
    async def _process_specialized_query(self, query: str) -> Dict[str, Any]:
        """Process career guidance queries."""
        query_lower = query.lower()
        
        if any(word in query_lower for word in ["career", "job", "work", "profession"]):
            content = "I can help you explore career opportunities! Based on your major and interests, I'll show you potential career paths, required skills, and job market trends. What field interests you most?"
            tools_used = ["find_career_paths"]
        
        elif any(word in query_lower for word in ["internship", "experience", "practice"]):
            content = "Internships are a great way to gain experience! I can recommend internship opportunities that align with your career goals and academic background."
            tools_used = ["recommend_internships"]
        
        elif any(word in query_lower for word in ["skills", "learn", "develop", "improve"]):
            content = "I'll analyze the skills needed for your target career and suggest courses or experiences to develop them. What career path are you considering?"
            tools_used = ["suggest_skill_development"]
        
        else:
            content = "I'm your career advisor! I can help you explore career paths, find internships, and develop the right skills for your future. What career questions do you have?"
            tools_used = []
        
        return {
            "content": content,
            "agent_type": self.agent_type.value,
            "tools_used": tools_used,
            "context_size": len(self.context.conversation_history)
        }

print("✅ Specialized agents implemented")

## Agent Coordinator: Intelligent Routing

Now let's create a coordinator that routes queries to the appropriate specialized agent:

In [ ]:
class AgentCoordinator:
    """Coordinates between specialized agents and routes queries appropriately."""
    
    def __init__(self, student_id: str):
        self.student_id = student_id
        
        # Initialize specialized agents
        self.agents = {
            AgentType.COURSE_EXPLORER: CourseExplorerAgent(AgentType.COURSE_EXPLORER, student_id),
            AgentType.ACADEMIC_PLANNER: AcademicPlannerAgent(AgentType.ACADEMIC_PLANNER, student_id),
            AgentType.CAREER_ADVISOR: CareerAdvisorAgent(AgentType.CAREER_ADVISOR, student_id)
        }
        
        # Query routing patterns
        self.routing_patterns = {
            AgentType.COURSE_EXPLORER: [
                "search", "find", "courses", "classes", "browse", "explore", 
                "details", "about", "information", "description"
            ],
            AgentType.ACADEMIC_PLANNER: [
                "plan", "degree", "graduation", "requirements", "prerequisites", 
                "next semester", "should take", "ready for", "timeline"
            ],
            AgentType.CAREER_ADVISOR: [
                "career", "job", "work", "profession", "internship", 
                "opportunities", "skills", "industry", "employment"
            ]
        }
    
    def route_query(self, query: str) -> AgentType:
        """Determine which agent should handle the query."""
        query_lower = query.lower()
        
        # Score each agent based on keyword matches
        agent_scores = {}
        
        for agent_type, keywords in self.routing_patterns.items():
            score = sum(1 for keyword in keywords if keyword in query_lower)
            if score > 0:
                agent_scores[agent_type] = score
        
        # Return agent with highest score, default to course explorer
        if agent_scores:
            return max(agent_scores.items(), key=lambda x: x[1])[0]
        else:
            return AgentType.COURSE_EXPLORER  # Default agent
    
    async def process_query(self, query: str) -> Dict[str, Any]:
        """Process query by routing to appropriate specialized agent."""
        # Route to appropriate agent
        target_agent_type = self.route_query(query)
        target_agent = self.agents[target_agent_type]
        
        # Process with specialized agent
        response = await target_agent.process_query(query)
        
        # Add routing information
        response["routed_to"] = target_agent_type.value
        response["routing_reason"] = self._get_routing_reason(query, target_agent_type)
        
        return response
    
    def _get_routing_reason(self, query: str, agent_type: AgentType) -> str:
        """Explain why query was routed to specific agent."""
        query_lower = query.lower()
        matched_keywords = [
            keyword for keyword in self.routing_patterns[agent_type] 
            if keyword in query_lower
        ]
        
        if matched_keywords:
            return f"Matched keywords: {', '.join(matched_keywords[:3])}"
        else:
            return "Default routing"
    
    def get_system_status(self) -> Dict[str, Any]:
        """Get status of all specialized agents."""
        status = {
            "student_id": self.student_id,
            "agents": {},
            "total_conversations": 0
        }
        
        for agent_type, agent in self.agents.items():
            agent_summary = agent.get_context_summary()
            status["agents"][agent_type.value] = agent_summary
            status["total_conversations"] += agent_summary["conversation_length"]
        
        return status

# Initialize the coordinator
coordinator = AgentCoordinator("test_student")

print("✅ Agent coordinator initialized")
print(f"📋 Available agents: {list(coordinator.agents.keys())}")

## Demonstration: Context Quarantine in Action

Let's see how context quarantine works by running different types of conversations:

In [ ]:
# Test context quarantine with different conversation types
print("🧪 Testing Context Quarantine")
print("=" * 60)

# Simulate different conversation flows
conversation_scenarios = [
    # Course exploration conversation
    {
        "name": "Course Exploration",
        "queries": [
            "I want to find machine learning courses",
            "Tell me more about CS401",
            "Are there any online AI courses?"
        ]
    },
    # Academic planning conversation
    {
        "name": "Academic Planning",
        "queries": [
            "Help me plan my computer science degree",
            "What courses should I take next semester?",
            "Can I take CS301 without CS201?"
        ]
    },
    # Career guidance conversation
    {
        "name": "Career Guidance",
        "queries": [
            "What careers are available in data science?",
            "I need internship recommendations",
            "What skills should I develop for AI jobs?"
        ]
    }
]

# Process each conversation scenario
for scenario in conversation_scenarios:
    print(f"\n🎭 Scenario: {scenario['name']}")
    print("-" * 40)
    
    for i, query in enumerate(scenario['queries'], 1):
        print(f"\n{i}. User: {query}")
        
        # Process query through coordinator
        response = await coordinator.process_query(query)
        
        print(f"   🤖 Agent: {response['routed_to']}")
        print(f"   📝 Response: {response['content'][:100]}...")
        print(f"   🔧 Tools: {response['tools_used']}")
        print(f"   📊 Context Size: {response['context_size']} messages")

print("\n" + "=" * 60)

## Context Isolation Analysis

Let's analyze how context quarantine maintains isolation between different conversation types:

In [ ]:
# Analyze context isolation
print("📊 Context Isolation Analysis")
print("=" * 50)

# Get system status
status = coordinator.get_system_status()

print(f"Student ID: {status['student_id']}")
print(f"Total Conversations Across All Agents: {status['total_conversations']}")
print("\n📋 Agent-Specific Context:")

for agent_name, agent_info in status['agents'].items():
    print(f"\n🤖 {agent_name.replace('_', ' ').title()}:")
    print(f"   Memory Namespace: {agent_info['memory_namespace']}")
    print(f"   Conversation Length: {agent_info['conversation_length']} messages")
    print(f"   Active Tools: {agent_info['active_tools']}")
    print(f"   Context Data: {agent_info['context_data_keys']}")

# Demonstrate context isolation benefits
print("\n💡 Context Quarantine Benefits:")
print("   ✅ Isolated Memory: Each agent maintains separate conversation history")
print("   ✅ Focused Tools: Agents only have access to relevant tools")
print("   ✅ Specialized Context: Domain-specific data doesn't contaminate other agents")
print("   ✅ Reduced Confusion: No irrelevant information in decision-making")

# Compare with non-quarantined approach
print("\n🔄 Comparison: Quarantined vs. Non-Quarantined")
print("\n📊 Without Quarantine (Single Agent):")
print("   ❌ All conversations mixed together")
print("   ❌ Course browsing affects academic planning")
print("   ❌ Career advice contaminated by course preferences")
print("   ❌ Large context window with irrelevant information")

print("\n📊 With Quarantine (Specialized Agents):")
print("   ✅ Conversations isolated by domain")
print("   ✅ Academic planning focused on requirements")
print("   ✅ Career advice based on career-specific context")
print("   ✅ Smaller, focused context windows")

## 🧪 Hands-on Exercise: Design Your Quarantine Strategy

Now it's your turn to experiment with context quarantine patterns:

In [ ]:
# Exercise: Create your own specialized agent
print("🧪 Exercise: Design Your Context Quarantine Strategy")
print("=" * 60)

# TODO: Create a new specialized agent for financial planning
class FinancialPlannerAgent(SpecializedAgent):
    """Specialized agent for tuition costs and financial planning."""
    
    def _setup_agent_capabilities(self):
        """Setup financial planning specific tools."""
        self.context.active_tools = [
            "calculate_tuition_cost",
            "check_financial_aid",
            "estimate_total_cost",
            "payment_plan_options"
        ]
        
        self.context.context_data = {
            "budget_constraints": {},
            "financial_aid_status": {},
            "cost_calculations": []
        }
    
    async def _process_specialized_query(self, query: str) -> Dict[str, Any]:
        """Process financial planning queries."""
        query_lower = query.lower()
        
        if any(word in query_lower for word in ["cost", "tuition", "fees", "price"]):
            content = "I can help you calculate tuition costs for your courses and degree program. Which courses are you planning to take?"
            tools_used = ["calculate_tuition_cost"]
        
        elif any(word in query_lower for word in ["financial aid", "scholarship", "grant", "loan"]):
            content = "Let me check your financial aid options and eligibility. I'll help you understand available scholarships, grants, and loan programs."
            tools_used = ["check_financial_aid"]
        
        elif any(word in query_lower for word in ["budget", "afford", "payment", "plan"]):
            content = "I can help you create a budget and payment plan for your education. Let's look at your total costs and payment options."
            tools_used = ["payment_plan_options"]
        
        else:
            content = "I'm your financial planning assistant! I can help you calculate costs, explore financial aid, and create payment plans. What financial questions do you have?"
            tools_used = []
        
        return {
            "content": content,
            "agent_type": self.agent_type.value,
            "tools_used": tools_used,
            "context_size": len(self.context.conversation_history)
        }

# Add the financial planner to your coordinator
coordinator.agents[AgentType.PREFERENCE_MANAGER] = FinancialPlannerAgent(AgentType.PREFERENCE_MANAGER, "test_student")
coordinator.routing_patterns[AgentType.PREFERENCE_MANAGER] = [
    "cost", "tuition", "fees", "price", "budget", "afford", 
    "financial aid", "scholarship", "payment", "loan"
]

# Test your new agent
financial_queries = [
    "How much will my computer science degree cost?",
    "What financial aid options are available?",
    "Can I afford to take 5 courses next semester?"
]

print("\n🧪 Testing Financial Planner Agent:")
for query in financial_queries:
    print(f"\n📝 Query: {query}")
    response = await coordinator.process_query(query)
    print(f"🤖 Routed to: {response['routed_to']}")
    print(f"📝 Response: {response['content'][:80]}...")

print("\n🤔 Reflection Questions:")
print("1. How does the financial planner maintain separate context from other agents?")
print("2. What happens when a query could match multiple agents?")
print("3. How would you handle cross-agent information sharing?")
print("4. What other specialized agents would be useful for your domain?")

print("\n🔧 Your Turn: Try These Modifications:")
print("   • Add more sophisticated routing logic")
print("   • Create agents for other domains (scheduling, social, etc.)")
print("   • Implement agent-to-agent communication")
print("   • Add memory sharing between related agents")

## Key Takeaways

From this exploration of context quarantine, you've learned:

### 🎯 **Core Concepts**
- **Context contamination** occurs when irrelevant information degrades agent performance
- **Specialized agents** maintain focused, domain-specific contexts
- **Memory isolation** prevents cross-contamination between conversation types
- **Intelligent routing** directs queries to the most appropriate agent

### 🛠️ **Implementation Patterns**
- **Agent specialization** with domain-specific tools and capabilities
- **Namespace isolation** using separate memory spaces
- **Coordinator pattern** for intelligent query routing
- **Context tracking** within each specialized domain

### 📊 **Performance Benefits**
- **Reduced context noise** improves decision quality
- **Faster processing** with smaller, focused contexts
- **Better tool selection** within specialized domains
- **Improved user experience** with domain-expert responses

### 🔄 **Architecture Advantages**
- **Scalability**: Easy to add new specialized agents
- **Maintainability**: Clear separation of concerns
- **Flexibility**: Agents can be developed and updated independently
- **Reliability**: Failures in one agent don't affect others

### 🚀 **Next Steps**
In the next notebook, we'll explore **Context Pruning** - how to intelligently remove irrelevant, outdated, or redundant information from your agent's memory to maintain optimal context quality.

The context quarantine system you've built provides the foundation for more sophisticated memory management techniques.

---

**Ready to continue?** Move on to `03_context_pruning.ipynb` to learn about intelligent memory cleanup and relevance filtering!